# Distance measures

It is often the case that that we wish to measure how close a an experimentally prepared quantum state is to the ideal. Or between an ideal quantum gate and it's experimental implementation. Distance measures between states or processes can be subtle.  We recomend thinking about the *operational intepretation* of each measure before using the measure. 


The following references are good starting points for further reading of the literature: 

*Quantum Computation and Quantum Information*  
Michael A. Nielsen & Isaac L. Chuang  
Cambridge: Cambridge University Press (2000)  
             
*Distinguishability and Accessible Information in Quantum Theory*  
Christopher A. Fuchs  
Ph.D. thesis, The University of New Mexico (1996)  
https://arxiv.org/abs/quant-ph/9601020  

*Distance measures to compare real and ideal quantum processes*  
Alexei Gilchrist, Nathan K. Langford, Michael A. Nielsen  
Phys. Rev. A 71, 062310 (2005)  
https://doi.org/10.1103/PhysRevA.71.062310  
https://arxiv.org/abs/quant-ph/0408063  

*Characterizing Quantum Gates via Randomized Benchmarking*  
Easwar Magesan, Jay M. Gambetta, and Joseph Emerson  
Phys. Rev. A 85, 042311 (2012)  
https://doi.org/10.1103/PhysRevA.85.042311  
https://arxiv.org/abs/1109.6887  


In [1]:
import numpy as np
import forest.benchmarking.operator_tools.random_operators as rand_ops
from forest.benchmarking.operator_tools.calculational import outer_product
import forest.benchmarking.distance_measures as dm

## Distance measures between quantum states

In [2]:
# some pure states 
psi1 = rand_ops.haar_rand_state(2)
rho_haar1 = outer_product(psi1,psi1)
psi2 = rand_ops.haar_rand_state(2)
rho_haar2 = outer_product(psi2,psi2)

# some mixed states
rho = rand_ops.bures_measure_state_matrix(2) 
sigma = rand_ops.bures_measure_state_matrix(2) 

The fidelity between $\rho$ and $\sigma$

$$F(\rho, \sigma) = \left(\operatorname{Tr} \sqrt{\sqrt{\rho} \sigma \sqrt{\rho}}\right)^2.$$  

When $\rho = |\psi\rangle \langle \psi|$ and $\sigma= |\phi\rangle \langle \phi|$ are pure states, the definition reduces to the squared overlap between the states: $F(\rho, \sigma)=|\langle\psi|\phi\rangle|^2$. Here it is easy to see that the fidelity is a probablity.

Be careful not to confuse this defintion with the **square root fidelity $\sqrt{F}$**. 

In [3]:
dm.fidelity(rho, sigma)

0.6520815389969493

In [4]:
print('Infidelity is 1 - fidelity:', dm.infidelity(rho, sigma))

Infidelity is 1 - fidelity: 0.3479184610030507


In [5]:
dm.trace_distance(rho, sigma)

0.3402863160096079

In [6]:
dm.bures_distance(rho, sigma)

0.6204584455548694

In [7]:
dm.bures_angle(rho, sigma)

0.6308682842487863

In [8]:
dm.hilbert_schmidt_ip(rho, sigma)

0.6386793328130461

In [9]:
qcb_exp, s_opt = dm.quantum_chernoff_bound(rho,sigma)
print('The the non-logarithmic quantum Chernoff bound is:', qcb_exp)
print('The s achieving the minimum qcb_exp is:', s_opt)

The the non-logarithmic quantum Chernoff bound is: 0.7424472339623657
The s achieving the minimum qcb_exp is: 0.677310028692339


Here we calculate the total variation distance (TVD) between the classical outcome distribuions associated with two random states in the Z basis.

In [10]:
Proj_zero = np.array([[1, 0], [0, 0]])

# Pr(0|rho) = Tr[ rho * Proj_zero ]
p = np.trace(rho_haar1 @ Proj_zero)
q = np.trace(rho_haar2 @ Proj_zero)

# Pr(Proj_one) = 1 - p or Pr(Proj_one) = 1 - q 
P = np.array([[p], [1-p]])
Q = np.array([[q], [1-q]])

dm.total_variation_distance(P,Q)

0.3898287392115576

The next two measures are not really measures between states; however you can think of them as a measure of how close (or how far, resp.) the given state is to a pure state.

In [11]:
print('Pure states have purity P = ', dm.purity(rho_haar1))
print('Mixed states have purity <=1. In this case P = ', np.round(dm.purity(rho),4), '\n')

print('Pure states have impurity L = 1 - Purity = ', dm.impurity(rho_haar1))
print('Mixed states have impurity >= 0. In this case L = ', np.round(dm.purity(rho),4))

Pure states have purity P =  1.0000000000000004
Mixed states have purity <=1. In this case P =  0.76 

Pure states have impurity L = 1 - Purity =  -4.440892098500626e-16
Mixed states have impurity >= 0. In this case L =  0.76


Dimensional renormalization makes the purity lay between [0,1].

The maximally mixed state has purity = 0, independent of dimension D.

In [12]:
# calculate purity WITH and WITHOUT dimensional renormalization
print(dm.purity(rho, dim_renorm=True))
print(dm.purity(rho, dim_renorm=False))

0.5200930246189173
0.7600465123094586


## Distance measures between quantum processes

The example is related to test cases borrowed from [qutip](https://github.com/qutip/qutip/blob/master/qutip/tests/test_metrics.py),
which were in turn generated using [QuantumUtils for MATLAB](https://goo.gl/oWXhO9) by C. Granade.

In [13]:
Id = np.asarray([[1, 0], [0, 1]])
Xd = np.asarray([[0, 1], [1, 0]])

from scipy.linalg import expm

# Define unitary
theta = 0.4
Ud = expm(-theta*1j*Xd/2)

# This unitary is:
# close to Id for theta small
# close to X for theta np.pi (up to global phase -1j)
print(Ud)

[[0.98006658+0.j         0.        -0.19866933j]
 [0.        -0.19866933j 0.98006658+0.j        ]]


**Process Fidelity between Pauli-Liouville matrices**

In [14]:
from forest.benchmarking.operator_tools import kraus2pauli_liouville

In [15]:
plio0 = kraus2pauli_liouville(Id) 
plio1 = kraus2pauli_liouville(Ud)
plio2 = kraus2pauli_liouville(Xd)

In [16]:
dm.process_fidelity(plio0, plio1)

0.9736869980009617

**Diamond norm distance between Choi matrices**

In [17]:
from forest.benchmarking.operator_tools import kraus2choi

In [18]:
choi0 = kraus2choi(Id) 
choi1 = kraus2choi(Ud)
choi2 = kraus2choi(Xd)

In [19]:
# NBVAL_SKIP
# our build environment has problems with cvxpy so we skip this cell

dnorm = dm.diamond_norm_distance(choi0, choi1)
print("This gate is close to the identity as the diamond norm is close to zero. Dnorm= ",dnorm)

WARN: m less than n, problem likely degenerate
This gate is close to the identity as the diamond norm is close to zero. Dnorm=  inf


In [20]:
# NBVAL_SKIP

dnorm = dm.diamond_norm_distance(choi0, choi2)
print("This gate is far from identity as diamond norm = ",dnorm)

WARN: m less than n, problem likely degenerate
This gate is far from identity as diamond norm =  inf


In [21]:
dm.watrous_bounds((choi0 - choi1)/2)

(0.3973386615901225, 1.58935464636049)